- capire come scrapare meglio user review counts
- (scrape link full cast);
- (creare funzione per cinematographer, writers, editor e composer);
- (creare funzione per scrapare people);
- join df, df2, (df3)
- pulizia dati (contare nulle, casting)
- nascondere api key
- aggiungere table of contents

# Project description

The goal of this project is to build a database of the 10 000 most popular feature films on IMDB. We'll define "popularity" as the number of ratings a film has received. 

The project will comprise the following steps:

- collecting the data through scraping and API requests;
- loading the data onto a PostgreSQL database;
- querying the database with SQL;
- visualizing the output of the queries

# Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time

# Data collection

IMDB doesn't have an official API. However, there are two unofficial APIs providing access to IMDB data: The Open Movie Database (OMDb) and The Movie Database (TMDB). In this project we'll rely on the latter as it doesn't enforce a rate limit. We're also going to supplement the API data with scraped data.

The TMDB API works by supplying the ID of the film(s) whose info we'd like to retrieve. The IDs consist of a double "t" followed by seven or eight digits. They can be found inside a film URL. For example, "tt0050782" is the ID for this film: https://www.imdb.com/title/tt0050782/. 

The first step will thus be retrieving the IDs of the 10 000 most popular films on IMDB, which can be found in this [playlist](https://www.imdb.com/search/keyword/?mode=detail&page=1&title_type=movie&ref_=kw_ref_rt_vt&num_votes=5000%2C&sort=num_votes,desc).

## Scraping the film IDs and the URLs

To scrape the film IDs of our 10 000 films we'll need to iterate a for loop over the first 200 pages in the playlist (each containing 50 films). The film ID is the value of the 'href' attribute inside h3 elements with the "lister-item-header" class. We'll write a simple regex to extract the film ID. We'll also scrape the entire URL: it's going to come in handy later when we do the scraping.

In [2]:
film_ids = []
film_urls = []

for i in range(1, 201):
  
  content = requests.get("https://www.imdb.com/search/keyword/?mode=detail&page=" + str(i) + "&title_type=movie&ref_=kw_ref_rt_vt&num_votes=5000%2C&sort=num_votes,desc").content
  
  soup = BeautifulSoup(content)

  for film in soup.find_all("h3", class_="lister-item-header"):

    link = film.find("a").get("href")
    film_id = re.findall(pattern = r"tt\d+", string = link)[0]
    film_ids.append(film_id)

    film_url = "https://www.imdb.com" + link
    film_urls.append(film_url)
    

## API calls

We'll now write a function to request the data from TMDB. The function takes a vector of film IDs and for each ID it sends a GET request to the API. If the status code is equal to 200 (i.e. the request has been successful) we'll append the data to a Pandas dataframe. Otherwise, we'll just append the film_id and we'll fill the remaining columns with null values.

In [3]:
def build_film_df(film_ids):

  df = pd.DataFrame(columns = ["id", "title", "release_date", "runtime", "country", "language", "genre", "studios", "budget", "revenue"])

  for film_id in film_ids:
      
      response = requests.get("https://api.themoviedb.org/3/movie/" + film_id + "?api_key=eb74f269207d7563d10da8190e228f61")

      if response.status_code == 200:  

        response_json = response.json()

        df = df.append({"id": response_json["imdb_id"],
                      "title": response_json["title"],
                      "release_date": response_json["release_date"],
                      "runtime": response_json["runtime"],
                      "country": ';'.join([country['name'] for country in response_json["production_countries"]]),
                      "language": ';'.join([language["english_name"] for language in response_json["spoken_languages"]]),
                      "genre": ';'.join([genre["name"] for genre in response_json["genres"]]),
                      "studios": ';'.join([company["name"] for company in response_json['production_companies']]),
                      "budget": response_json['budget'],
                      "revenue": response_json["revenue"]}, 
                       ignore_index = True)
        
      else:
        df = df.append({"id": film_id}, ignore_index = True)

  return df

In [4]:
df = build_film_df(film_ids)
df.head()

,id,title,release_date,runtime,country,language,genre,studios,budget,revenue
0,tt0111161,The Shawshank Redemption,1994-09-23,142,United States of America,English,Drama;Crime,Castle Rock Entertainment,25000000,28341469
1,tt0468569,The Dark Knight,2008-07-14,152,United Kingdom;United States of America,English;Mandarin,Drama;Action;Crime;Thriller,DC Comics;Legendary Pictures;Syncopy;Isobel Gr...,185000000,1004558444
2,tt1375666,Inception,2010-07-15,148,United Kingdom;United States of America,English;Japanese,Action;Science Fiction;Adventure,Legendary Pictures;Syncopy;Warner Bros. Pictures,160000000,825532764
3,tt0137523,Fight Club,1999-10-15,139,Germany;United States of America,English,Drama,Regency Enterprises;Fox 2000 Pictures;Taurus F...,63000000,100853753
4,tt0109830,Forrest Gump,1994-07-06,142,United States of America,English,Comedy;Drama;Romance,Paramount;The Steve Tisch Company,55000000,677387716


Let's have a look at the null values

In [5]:
df[df['title'].isnull()]

,id,title,release_date,runtime,country,language,genre,studios,budget,revenue
5241,tt9573980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9128,tt8075016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9978,tt13423846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Scraping additional data

TMDB doesn't provide data about the people who worked on a film (directors, writer, actors etc.) and its rating data is inaccurate. We'll thus integrate the data we just got by scraping some more information.

### Custom functions

To implement sound software engineering principles we'll scrape the data by building a function for each type of data we need.

#### Film ID

In [6]:
def scrape_film_id(soup):
  
  return soup.find("meta", {"property": "imdb:pageConst"}).get("content")

#### Directors

In [25]:
def scrape_director(soup):
    
    a_tags = soup.find_all(href = re.compile("tt_ov_dr"), class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
    
    directors = list(set([a.text for a in a_tags]))
    
    return ';'.join(directors)

#### Writers

In [23]:
def scrape_writer(soup):
    
  a_tags = soup.find_all(href = re.compile("tt_ov_wr"), class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")
  
  writers = list(set([a.text for a in a_tags]))

  return ';'.join(writers)

#### Actors

In [19]:
def scrape_actors(soup):
    
    a_tags = soup.find_all(href = re.compile("tt_ov_st"), class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link")

    actors = list(set([a.text for a in a_tags]))

    return ';'.join(actors)

#### IMDB average rating

In [8]:
def scrape_imdb_rating(soup):
  
  pattern = r'(?<="ratingValue":)[\d.]+(?=},"contentRating")'
  string = str(soup.find("script", {"type": "application/ld+json"}))

  return re.findall(pattern = pattern,
                    string = string)[0]

#### IMDB rating count

In [9]:
def scrape_rating_count(soup):

  pattern = r'(?<="ratingCount":)[\d.]+'
  string = str(soup.find("script", {"type": "application/ld+json"}))

  return re.findall(pattern = pattern,
                    string = string)[0]

#### Metascore

In [17]:
def scrape_metascore(soup):

  return soup.find("span", class_="score-meta").text if soup.find("span", class_="score-meta") else np.nan

#### User reviews count

In [11]:
def scrape_user_review_count(soup):
  
  pattern = r'(?<="total":)\d+(?=,"__typename":"ReviewsConnection"},"criticReviewsTotal":)'
  
  string = str(soup.find("script", {'id': '__NEXT_DATA__'}))
  
  return re.findall(pattern = pattern, string = string)[0] if re.findall(pattern = pattern, string = string) else np.nan

#### Critic reviews count

In [12]:
def scrape_critic_review_count(soup):
    
    string = list(filter(lambda x: 'Critic' in str(x), soup.find_all("span", class_="three-Elements")))[0].text
    
    return re.findall(r'\d+', string)[0]

#### Color

In [13]:
def scrape_color(soup):
    
    pattern = r'(?<="text":")\w+(?=","attributes":\[\],"__typename":"Coloration"})'
    
    string = str(soup.find("script", {"type": "application/json"}))
    
    return re.findall(pattern = pattern, string = string)[0] if re.findall(pattern = pattern, string = string) else np.nan

#### Aspect ratio

In [14]:
def scrape_aspect_ratio(soup):
    
    pattern = r'(?<="aspectRatio":")[\d.\s:]+'
    
    string = str(soup.find("script", {"type": "application/json"}))
    
    return re.findall(pattern = pattern, string = string)[0] if re.findall(pattern = pattern, string = string) else np.nan

### Scraping

If we send 10 000 requests at once we're likely going to run into problems with IMDB, so let's split the film_urls list into 5 equally-sized chunks and let's scrape each chunk at a time by waiting five minutes before moving to the next one.

In [15]:
def build_scraped_df(urls):

  df = pd.DataFrame(columns = ["id", "director", "imdb_rating", "imdb_rating_count", "metascore", 
                               "user_review_count", "critic_review_count", "color", "aspect_ratio"])

  chunks = np.array_split(urls, 5)

  for chunk in chunks:
    for url in chunk:
      content = requests.get(url).content
      soup = BeautifulSoup(content)

      df = df.append({"id": scrape_film_id(soup),
                      "director": scrape_director(soup),
                      "imdb_rating": scrape_imdb_rating(soup),
                      "imdb_rating_count": scrape_rating_count(soup),
                      "metascore": scrape_metascore(soup),
                      "user_review_count": scrape_user_review_count(soup),
                      "critic_review_count": scrape_critic_review_count(soup),
                      "color": scrape_color(soup),
                      "aspect_ratio": scrape_aspect_ratio(soup)}, 
                      ignore_index = True)

    time.sleep(60*5)

  return df


In [16]:
df2 = build_scraped_df(film_urls)

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
df2.head()

Counting the null values for each column

In [ ]:
test.apply(lambda x: x.isnull().sum())

Is each id unique?

In [ ]:
test['id'].duplicated().sum()

#### Full cast

In [ ]:
def scrape_actors(soup):
  
  actors = [img.get("alt") for img in soup.find("table", class_="cast_list").find_all("img")]
  return ';'.join(actors)

# Load the data onto a database